In [1]:
## Load in the required packages

library(quantmod) 
library(tidyverse) 
library(reshape2) 
library(lubridate) 
library(glmnet)
library(ISLR) 
library(cowplot) 

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::first()   mask

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-0"><strong><em>Description</em></strong></a></span></li><li><span><a href="#Read-in-Functions" data-toc-modified-id="Read-in-Functions-1"><strong><em>Read in Functions</em></strong></a></span><ul class="toc-item"><li><span><a href="#Define-Start-and-End-Dates" data-toc-modified-id="Define-Start-and-End-Dates-1.1"><strong><em>Define Start and End Dates</em></strong></a></span></li><li><span><a href="#Normalize_Data" data-toc-modified-id="Normalize_Data-1.2"><strong><em>Normalize_Data</em></strong></a></span></li><li><span><a href="#Read_FRED_Data" data-toc-modified-id="Read_FRED_Data-1.3"><strong><em>Read_FRED_Data</em></strong></a></span></li><li><span><a href="#Read_Yahoo_Data" data-toc-modified-id="Read_Yahoo_Data-1.4"><strong><em>Read_Yahoo_Data</em></strong></a></span></li></ul></li><li><span><a href="#Read-in-the-Data" data-toc-modified-id="Read-in-the-Data-2"><strong><em>Read in the Data</em></strong></a></span></li><li><span><a href="#Summarize-the-Data" data-toc-modified-id="Summarize-the-Data-3"><strong><em>Summarize the Data</em></strong></a></span><ul class="toc-item"><li><span><a href="#Create-the-Lasso-Regression-Model" data-toc-modified-id="Create-the-Lasso-Regression-Model-3.1"><strong><em>Create the Lasso Regression Model</em></strong></a></span></li></ul></li></ul></div>

## ***Description*** 

We are looking to predict the S&P 500 returns using a min of 10 features. 
*******
Start by downloading the data
******* 
Define the outcome variable as the next period return of the S&P 500 
******
Find 10 feature variables that can be used as the predictors of the S&P 500 returns 
****** 
Build a Ridge Regression and a Lasso Regression using R's glmnet package 
Calibrate the models by selecting the lambda values that minimize the out of sample MSE 
***** 
Summarize the key insights found from the two different models 


## ***Read in Functions***

We are going to begin by writing functions responsible for reading in the data 

### ***Define Start and End Dates***

Begin by defining the start and end dates we are going to use. 
This defines the training data range. 


In [2]:
train_start <- as.Date('2000-01-01') 
train_end <-  Sys.Date()

### ***Normalize_Data*** 

In [3]:
Normalize_Data <- function(x){
    ## Normalize the data in order to remove any effects
    ## The units of measurement may have on the result 
    
    return((x - mean(x)) / sd(x) ) 
}

### ***Read_FRED_Data***
We are going to create a function that read in data from FRED

In [4]:
Read_FRED_Data <- function(start = train_start, 
                           end = train_end, ticker){
    
    ## Read in the fred data
    df <- getSymbols(ticker, src = 'FRED', auto.assign = F, from = start, 
                    to = end)  
    
    ## Find the dates and the values 
    df_dates <- index(df) 
    ## Convert data into dataframe
    data <- as_tibble(df) 
    ## Rename the columns so we can more easily handle it in tibble 
    names(data) <- c('value') 
 
    
    data <- data %>% 
        ## Add the dates to the dataframe 
        add_column(date = df_dates) %>% 
    
        ## Normalize the data in order to remove any effect different
        ## units of measurement may have on correlation
        mutate(value = Normalize_Data(value)) %>%  
    
        ## Group the data into 3 month periods 
        ## Take the first datapoint from each period 
        ## So we can see the quarterly data intervals for predictions 
        group_by(quarter = quarter(date, with_year = T) ) %>% 
        summarize(value = first(value)) %>% 
        filter(quarter > quarter(train_start, with_year = T)) 
    
    ## Rename the columns back to original ticker name and date for easy formatting 
    names(data) <- c('date', ticker) 
        
    return(data) 
    
}

### ***Read_Yahoo_Data*** 

In [5]:
Read_Yahoo_Data <- function(start = train_start, 
                           to = train_end, 
                           ticker){
    
    ## Read in ticker symbols for the s&p500 data 
    data <- getSymbols('^GSPC', src = 'yahoo', 
                    from = train_start, to = train_end, 
                    auto.assign = F, period = 'monthly') 
    
    ## Read in the dates
    dates <- index(data) 
    
    ## Convert to a dataframe 
    df <- as_tibble(data) 
    ## Rename all the columns for easy access and add the date
    names(df) <- c('open', 'high', 'low', 'close', 'volume', 'adjusted') 
    df$date <- dates
    
    ## We only need the date and the adjusted close columns 
    df <- df[,c('date', 'adjusted')] %>% 
    
        ## Normalize the adjusted closing data 
        mutate(adjusted = Normalize_Data(adjusted) ) %>% 
        
        ## Format the date so that its broken up into quarters
        ## and then group by quarter 
        ## Taking the first value of each quarter 
        group_by(date = quarter(date, with_year = T) ) %>% 
        summarize(adjusted = first(adjusted)) %>% 
        filter(date > quarter(train_start, with_year = T)) 
        
    
    return(df)
}   



## ***Read in the Data*** 

We need to find 10 different independent variables that can predict the future outcome of the S&P_500. We will combine them together in a single dataframe so we can then run the linear regressions

**** 
We are going to look at the following dependent variables: 
1. Unemployment Rate 
2. Real Personal Consumption Expenditure
3. Real Gross Domestic Product Per Capita 
4. Cash Assets, All Commercial Banks
5. Corporate Profits After Tax
6. Total Population 
7. Real Net Exports of goods and services 
8. Personal Consumption expenditures per capita
9. M2 Money Stock (M1 + time deposits) 
10. VIX (volatility index) from chicago board options exchange


In [8]:
## Read in the ticker symbols for all the fred data
fred_data <- c('UNRATE', ## unemployment rate
               'CPIAUCNS', ## consumer pricing index for all urban consumers
               'A939RX0Q048SBEA', # Real Gross Domestic Product Per Capita
               'CASACBQ158SBOG', # Cash Assets, all commercial banks
               'CP', # Corporate Profits After Tax 
               'B230RC0Q173SBEA', ## Total Population
               'NETEXC', ## Real Net Exports of goods and services
               'A794RC0Q052SBEA',  ## Personal Consumption expenditures per capita
               'M2' ## M2 Money Stock 
              )
feature_names <- c('date', 
                    'unemployment', 
                    'consumer_pricing_index', 
                    'gdp_per_capita', 
                    'cash_assets_in_commercial_banks', 
                    'corporate_profits_after_tax', 
                    'total_population', 
                    'net_exports_goods_services', 
                    'personal_consumption_expenditures_per_capita', 
                    'm2_money_stock', 
                    'cboe_vix', 'sp500')  

In [9]:
## Create a dataframe to store all the FRED Data. Make sure the dates match up
features <- tibble(date = numeric())

for (i in fred_data){
    ## Join the dataframe to append the FRED data
    ## Match them up with the correct dates 

    
    features <- features %>% 
        full_join(Read_FRED_Data(ticker = i))
        ## rename column
    

}

## Read in sp500 data
sp500 <- Read_Yahoo_Data(ticker = '^GSPC') %>% rename(sp500 = adjusted) 

## Add in the VIX data from Yahoo Data 
features <- features %>% 
    full_join(Read_Yahoo_Data(ticker = '^VIX')) %>% rename(vix = adjusted) %>% 
    full_join(sp500) 

## Rename the Data columns 
names(features) <- feature_names
## drop the null values 
features <- features %>% drop_na()
head(features) 

Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"
Joining, by = "date"


date,unemployment,consumer_pricing_index,gdp_per_capita,cash_assets_in_commercial_banks,corporate_profits_after_tax,total_population,net_exports_goods_services,personal_consumption_expenditures_per_capita,m2_money_stock,cboe_vix,sp500
2000.2,-1.1531259,1.128794,0.9404985,-0.11482788,0.018749563,0.7880232,-0.5397744,0.6628294,-0.4123912,-0.2287531,-0.2287531
2000.3,-1.0358951,1.147679,0.9358062,-0.40675930,0.008336457,0.8019429,-0.6450971,0.6861359,-0.4031681,-0.2816918,-0.2816918
2000.4,-1.0945105,1.162787,0.9479768,-0.31773057,-0.026018958,0.8155360,-0.6818328,0.7065750,-0.3822369,-0.3300967,-0.3300967
2001.1,-0.9186643,1.176636,0.9303808,-0.04443307,0.033178902,0.8273506,-0.6470369,0.7217940,-0.3599438,-0.5523727,-0.5523727
2001.2,-0.8014335,1.199297,0.9422581,-0.42953409,0.069708711,0.8394918,-0.6020204,0.7303226,-0.3248912,-0.7520374,-0.7520374
2001.3,-0.6842027,1.206851,0.9191633,0.43383756,-0.003955261,0.8530304,-0.6885395,0.7328958,-0.3055131,-0.6200175,-0.6200175


In [ ]:
## Plot FRED Data 
dependent_features_plot <- ggplot(data = melt(features, id = c('date')), 
       aes(x = date, y = value, col = variable)) + 
        geom_line() + 
        labs(x = 'Date', y = 'Normalized FRED Data', 
            colours = 'Variables', 
            title = 'Plot Normalized FRED Data Over Time')

sp500_plot <- ggplot(data = sp500, aes(x = date, y = sp500)) + geom_line() + 
    labs(x = 'Date', y = 'SP500 Adjusted Close', title = 'SP500 Adjusted Close Over Time') 

## Plot the FRED data over time as well as the SP500 over time 
plot_grid(dependent_features_plot, sp500_plot, ncol = 1) 

## ***Summarize the Data*** 

Now we have all the data we need

In [30]:
summary <- tibble(names = names(features), 
            std = sapply(features, sd), 
            mean = sapply(features, mean), 
            min = sapply(features, min), 
            max = sapply(features, max), 
             p.25 = 1, 
             p.50 = 1, 
             p.75 = 1)

v1 <- c(0.25,0.50,0.75)
summary[,c(6:8)] <- t(sapply(features, function(x) quantile(x,probs = v1)))

## Remove date
summary <- std[-1,]
summary

names,std,mean,min,max,p.25,p.50,p.75
consumer_pricing_index,0.3276025,1.70598926,1.12879393,2.2342012,1.4246569718,1.7212880,1.96522708
gdp_per_capita,0.2438614,1.30712897,0.91916326,1.8071784,1.1500750699,1.2946928,1.49842234
cash_assets_in_commercial_banks,1.4938156,0.11958583,-0.85397339,11.5727812,-0.3839845066,-0.1531309,0.08445155
corporate_profits_after_tax,0.7557202,1.43583656,-0.02978737,2.5753326,0.9502640916,1.6227884,2.05258601
total_population,0.2638900,1.26574322,0.78802321,1.6657837,1.0379978266,1.2918017,1.49664609
net_exports_goods_services,0.4636360,-1.09284668,-2.07504892,-0.2504139,-1.5095908993,-1.1488013,-0.65619856
personal_consumption_expenditures_per_capita,0.4367802,1.39982197,0.66282938,2.2104675,1.0806181437,1.3445254,1.74365791
m2_money_stock,0.8016129,0.69662549,-0.41239123,2.8357430,-0.0002008239,0.4986175,1.32836848
cboe_vix,0.9068680,-0.06998955,-1.23854218,2.3170161,-0.7481101864,-0.3923502,0.55718341
sp500,0.9068680,-0.06998955,-1.23854218,2.3170161,-0.7481101864,-0.3923502,0.55718341


### ***Create the Lasso Regression Model*** 

In [56]:
model_features <- 
x <- model.matrix(sp500~., features)[,3:12]
head(x) 

unemployment,consumer_pricing_index,gdp_per_capita,cash_assets_in_commercial_banks,corporate_profits_after_tax,total_population,net_exports_goods_services,personal_consumption_expenditures_per_capita,m2_money_stock,cboe_vix
-1.1531259,1.128794,0.9404985,-0.11482788,0.018749563,0.7880232,-0.5397744,0.6628294,-0.4123912,-0.2287531
-1.0358951,1.147679,0.9358062,-0.40675930,0.008336457,0.8019429,-0.6450971,0.6861359,-0.4031681,-0.2816918
-1.0945105,1.162787,0.9479768,-0.31773057,-0.026018958,0.8155360,-0.6818328,0.7065750,-0.3822369,-0.3300967
-0.9186643,1.176636,0.9303808,-0.04443307,0.033178902,0.8273506,-0.6470369,0.7217940,-0.3599438,-0.5523727
-0.8014335,1.199297,0.9422581,-0.42953409,0.069708711,0.8394918,-0.6020204,0.7303226,-0.3248912,-0.7520374
-0.6842027,1.206851,0.9191633,0.43383756,-0.003955261,0.8530304,-0.6885395,0.7328958,-0.3055131,-0.6200175
